# Benchmark

Description du benchmark

Le modèle de référence utilisé dans ce challenge utilise un modèle linéaire multiple. Chaque nœud est défini par l’intersection d’un numéro de train et un identifiant de gare. Nous utilisons un simple modèle linéaire avec au plus un voisinage p = q = 1.

$Y_{k, s} = \beta^0_{k, s} + \beta^{1, 0}_{k, s} y_{k-1, s} + \beta^{0, 1}_{k, s} y_{k, s-1} + \varepsilon_{k, s}$

quand le voisinage existe. Avec la syntaxe de R, cela donne :

lm(p0q0 ~ p1q0 + p0q1, data = dataByNode)

> Ici les deux variables qui nous intéressent sont p1q0, p0q1 et on fait une régression linéaire pour chaque couple (k, s)

> Dans ce modèle ultra simplifié, on remplace arbitrairement tous les NaN par des 0



In [1]:
from utils import *
_, _, df, _ = get_data()

In [2]:
# trains et  stations
# pour faire une régression linéaire pour chaque couple (k, s)
trains = df.train.unique()

# Comme un train ne déssert pas toutes les gares
# les stations sont une liste indexée par le trains
# stations est un dictionnaire qui a un train associe la liste des stations desservies
# stations[20] = ['AE', 'AI', 'AJ', 'AK', 'AL', 'AO', 'AQ', 'BB', 'BC']
stations = {k:list(df.loc[df.train==k].station.unique()) for k in trains}


In [3]:
# Dans ce modèle ultra simplifié, on remplace arbitrairement 
# tous les NaN par des 0
df.fillna(0,inplace=True)

In [4]:
# En phase d'évaluation des performances
chosen_columns = ['train', 'station','p1q0', 'p0q1']
_, _, df_train, _, _, df_test = split_data(df, chosen_columns)

In [5]:
# Nombre de paires (k, s) possibles
print('Nombre de paires (k, s) possibles: ',
      sum([len(stations[k]) for k in trains]))

Nombre de paires (k, s) possibles:  541


In [6]:
# Entraînement d'un modèle pour chaque (k,s)

from sklearn import linear_model

model = {}
dataset = {}

for k in trains:
    for s in stations[k]:
        df_train_ks = df_train.query('train==@k and station==@s')
        df_test_ks = df_test.query('train==@k and station==@s')

        x_train_ks = df_train_ks[['p1q0', 'p0q1']]
        y_train_ks = df_train_ks['p0q0']
        x_test_ks = df_test_ks[['p1q0', 'p0q1']]
        y_test_ks = df_test_ks['p0q0']
        dataset[(k,s)] = x_train_ks, y_train_ks, x_test_ks, y_test_ks

for k in trains:
    for s in stations[k]:
        x, y, _, _ = dataset[(k,s)]
        linear_classifier = linear_model.LinearRegression()
        linear_classifier.fit(x, y)
        model[(k,s)] = linear_classifier


In [7]:
# Evaluation du modèle

from sklearn.metrics import mean_absolute_error

errors = []

for k in trains:
    for s in stations[k]:
        _, _, x_test, y_test = dataset[(k,s)]

        linear_classifier = model[(k,s)]
        y_pred = linear_classifier.predict(x_test)
        errors.append(mean_absolute_error(y_test, y_pred))

from statistics import fmean
fmean(errors)*100
# Objectif 1.3

1.2009279492967329

# Proposition de solution

## Entraînement

In [8]:
_, _, df, x_test = get_data()
# trains et  stations
# pour faire une régression linéaire pour chaque couple (k, s)
trains = df.train.unique()

# Comme un train ne déssert pas toutes les gares
# les stations sont une liste indexée par le trains
stations = {k:list(df.loc[df.train==k].station.unique()) for k in trains}

df.fillna(0,inplace=True)

In [9]:
# Entraînement d'un modèle pour chaque (k,s)
from sklearn import linear_model

model = {}

for k in trains:
    for s in stations[k]:
        df_ks = df.query('train==@k and station==@s')
        x = df_ks[['p1q0', 'p0q1']]
        y = df_ks['p0q0']
        linear_classifier = linear_model.LinearRegression()
        linear_classifier.fit(x, y)
        model[(k,s)] = linear_classifier


## Prédictions

In [10]:
# Make predictions using the testing set
x_test.fillna(0,inplace=True)


In [16]:
y_pred = []
for row in x_test.itertuples():
    # on récupère le bon modèle pour ce train à cette station
    linear_classifier = model[(row.train, row.station)]
    df_row = pd.DataFrame(np.array([[row.p1q0, row.p0q1]]), columns=['p1q0', 'p0q1'])

    #infer from model
    infered_value = linear_classifier.predict(df_row)[0]
    y_pred.append(infered_value)


In [17]:
assert len(y_pred) == len(x_test)

In [ ]:
create_submission_csv(y_pred, 'y_benchmark')

score public 0,0131 (Ze benchmark)